In [2]:
#OL-Opx-Spl

#EOF
import thermoengine as thermo
from thermoengine import phases
from thermoengine import model
import numpy as np
import math
import ipywidgets as widgets
from scipy import optimize as optim
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Accordion, HBox, VBox, Label
from ipywidgets import Button, Output
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))


def create_expanded_button(description, button_style):
    """Helper function to create a button widget with a description and style."""
    button = Button(description=description, button_style=button_style)
    button.layout.height = 'auto'
    button.layout.width = 'auto'
    return button



# Create the main widget
window = widgets.VBox()

# Create a list of column names
column_names = ['Spinel', 'Orthopyroxene', 'Olivine']

# Create a list of empty strings to store the data
data = [['' for _ in range(3)] for _ in range(10)]

# Create a function to update the data list when the user clicks the "Submit" button
def update_data(b):
      for i in range(10):
    # Get the data from each entry widget and update the corresponding element in the data list
        data[i][0] = entries[i][0].value
        data[i][1] = entries[i][1].value
        data[i][2] = entries[i][2].value

# Create a function to clear the data when the user clicks the "Clear" button
def clear_data(b):
      for i in range(10):
        # Clear the data in each entry widget and update the corresponding element in the data list
        entries[i][0].value = ''
        entries[i][1].value = ''
        entries[i][2].value = ''
        data[i][0] = ''
        data[i][1] = ''
        data[i][2] = ''

# Create a label for each column
labels = []
for i, name in enumerate(column_names):
      label = widgets.Label(value=name)
      labels.append(label)
  
# Create a list of row names
row_names = ["SiO2", "TiO2", "Al2O3", "Cr2O3", "FeO", "MnO", "MgO", "CaO", "Fe2O3-spinel/ Na2O-orthopyroxene/ P2O5-olivine", "V2O3-spinel only, ignore other entry boxes"]

# Create a label for each row
row_labels = []
for i in range(10):
  rlabel = widgets.Label(value=row_names[i])
  row_labels.append(rlabel)

# Create an entry widget for each cell in the grid
entries = []
for i in range(10):
  row = []
  for j in range(3):
    entry = widgets.Text(placeholder='Enter value')
    row.append(entry)
  entries.append(row)

for i in range(10):
  for j in range(3):
    entries[i][j].placeholder = column_names[j]


# Create a grid to hold the entry widgets
grid = widgets.VBox(children=[], layout=widgets.Layout(grid_template_columns="repeat(4, 1fr)", grid_template_rows="repeat(11, 1fr)"))

# Create a "Clear" button
clear_button = widgets.Button(description="Clear", button_style='warning')
clear_button.on_click(clear_data)

# Create a "Continue" button
continue_button = widgets.Button(description="Continue", button_style='info')


# Add the row labels and entries to the grid
for i in range(10):
  # Add the row label to the first column of the current row
  grid.children = (*grid.children, widgets.Label(value=row_names[i]))
  # Add the entries to the remaining columns of the current row
  for j in range(3):
    grid.children = (*grid.children, entries[i][j])

#Entry widget for pressure 
entry = widgets.Text(description='P in bars:')

#List of empty strings to store the data
pressure_data = ['']

def update_pressure_data(b):
  # Get the value from the entry widget
  pressure_data[0] = entry.value
  # Update the pressure_data list with the entry value
  pressure_data[0] = entry.value

# Set up a callback function to update the data list when the button is clicked
continue_button.on_click(update_pressure_data)

# Add the grid and buttons to the main widget
children = [entry] + [grid] + [continue_button] + [clear_button]
window.children = children

# Create the FloatText widget
y_widget = widgets.FloatText(description='T°C', disabled=True, layout=widgets.Layout(width='200px', height='50px'))

# Create the FloatText widget
x_widget = widgets.FloatText(description='log fO2', disabled=True, layout=widgets.Layout(width='200px', height='50px'))

# Create the FloatText widget
z_widget = widgets.FloatText(description='dQFM fO2', disabled=True, layout=widgets.Layout(width='200px', height='50px'))

# Create the FloatText widget
aa_widget = widgets.FloatText(description='dIW fO2', disabled=True, layout=widgets.Layout(width='200px', height='50px'))

# Create the FloatText widget
ab_widget = widgets.FloatText(description='dNNO fO2', disabled=True, layout=widgets.Layout(width='200px', height='50px'))



# Create a VBox widget to hold the x and y widgets
result_box = widgets.VBox([y_widget, x_widget, z_widget, aa_widget, ab_widget])

print("Welcome! This notebook calculates the oxygen fugacity of a sample using the olivine-orthopyroxene-spinel geo-oxybarometer, and the temperature of Fe-Mg equilibration between olivine and spinel using the olivine-spinel geothermometer. To use this notebook, enter the pressure (in bars) of sample formation, as well as the wt.% values of the listed oxide phases for spinel, orthopyroxene, and olivine.")
print()
print("The oxygen fugacity results include:")
print()
print("logfO2 = absolute log fO2")
print()
print("dQFM fO2 = fO2 relative to the Quartz-Fayalite-Magnetite buffer, with values from Frost (1991)")
print()
print("dIW fO2 = fO2 relative to the Iron-Wustite buffer, with values from Euster and Wones (1962)")
print()
print("dNNO fO2 = fO2 relative to the Nickle-Nickle-Oxide buffer, with values from Huebner and Sato (1970)")
print()
print("The geothermometry and geobarometry calculations in this notebook are based on work by Jianping et al. (1995), Myers and Eugster (1983), Sack and Ghiorso (1989, 1991a, 1991b), Wood (1990), Wood and Nell (1991). For a review on oxybarometers, including the olivine-orthopyroxene-spinel barometer used here, check out the paper by Herd (2008). References to the corresponding papers can be found at the bottom of this screen.")
print()
print("To cite this notebook in publications please use the following citation: [will be added once final version is ready and Zenodo citation created]")
print()
print("If you encounter issues using this notebook, or have suggestions for how it could be improved, please email benaroya@ualberta.ca")
print()
# Create a function to be called when the "Continue" button is clicked

title = create_expanded_button('Olivine-Pyroxene-Spinel geobarometer', 'info')
display(title)

def on_continue_clicked(b):
    update_pressure_data(b)
    #SPINEL CALCULATIONS
    spl_SiO2 = entries[0][0].value
    if spl_SiO2:
        spl_SiO2 = float(spl_SiO2)
        spl_SiO2_mole = spl_SiO2 / 60.0843
        spl_SiO2_cations = spl_SiO2_mole * 1
        spl_SiO2_oxygens = spl_SiO2_mole * 2
        
    spl_TiO2 = entries[1][0].value
    if spl_TiO2:
        spl_TiO2 = float(spl_TiO2)
        spl_TiO2_mole = spl_TiO2 / 78.8658
        spl_TiO2_cations = spl_TiO2_mole * 1   
        spl_TiO2_oxygens = spl_TiO2_mole * 2

    spl_Al2O3 = entries[2][0].value
    if spl_Al2O3:
        spl_Al2O3 = float(spl_Al2O3)
        spl_Al2O3_mole = spl_Al2O3 / 101.961276
        spl_Al2O3_cations = spl_Al2O3_mole * 2   
        spl_Al2O3_oxygens = spl_Al2O3_mole * 3

    spl_Cr2O3 = entries[3][0].value
    if spl_Cr2O3:
        spl_Cr2O3 = float(spl_Cr2O3)
        spl_Cr2O3_mole = spl_Cr2O3 / 151.9904
        spl_Cr2O3_cations = spl_Cr2O3_mole * 2   
        spl_Cr2O3_oxygens = spl_Cr2O3_mole * 3
    
    spl_FeO = entries[4][0].value
    if spl_FeO:
        spl_FeO = float(spl_FeO)
        spl_FeO_mole = spl_FeO / 71.8444
        spl_FeO_cations = spl_FeO_mole * 1   
        spl_FeO_oxygens = spl_FeO_mole * 1

    spl_MnO = entries[5][0].value
    if spl_MnO:
        spl_MnO = float(spl_MnO)
        spl_MnO_mole = spl_MnO / 70.937449
        spl_MnO_cations = spl_MnO_mole * 1   
        spl_MnO_oxygens = spl_MnO_mole * 1

    spl_MgO = entries[6][0].value
    if spl_MgO:
        spl_MgO = float(spl_MgO)
        spl_MgO_mole = spl_MgO / 40.3044
        spl_MgO_cations = spl_MgO_mole * 1   
        spl_MgO_oxygens = spl_MgO_mole * 1   

    spl_CaO = entries[7][0].value
    if spl_CaO:
        spl_CaO = float(spl_CaO)
        spl_CaO_mole = spl_CaO / 56.0774
        spl_CaO_cations = spl_CaO_mole * 1   
        spl_CaO_oxygens = spl_CaO_mole * 1   

    spl_Fe2O3 = entries[8][0].value
    if spl_Fe2O3:
        spl_Fe2O3 = float(spl_Fe2O3)
        spl_Fe2O3_mole = spl_Fe2O3 / 159.6882
        spl_Fe2O3_cations = spl_Fe2O3_mole * 2   
        spl_Fe2O3_oxygens = spl_Fe2O3_mole * 3

    spl_V2O3 = entries[9][0].value
    if spl_V2O3:
        spl_V2O3 = float(spl_V2O3)
        spl_V2O3_mole = spl_V2O3 / 149.883
        spl_V2O3_cations = spl_V2O3_mole * 2   
        spl_V2O3_oxygens = spl_V2O3_mole * 3
        
    spl_Na2O = 0.0
    
    #ORTHOPYROXENE CALCULATIONS
    opx_SiO2 = entries[0][1].value
    if opx_SiO2:
        opx_SiO2 = float(opx_SiO2)
        opx_SiO2_mole = opx_SiO2 / 60.0843
        opx_SiO2_cations = opx_SiO2_mole * 1
        opx_SiO2_oxygens = opx_SiO2_mole * 2
        
    opx_TiO2 = entries[1][1].value
    if opx_TiO2:
        opx_TiO2 = float(opx_TiO2)
        opx_TiO2_mole = opx_TiO2 / 78.8658
        opx_TiO2_cations = opx_TiO2_mole * 1   
        opx_TiO2_oxygens = opx_TiO2_mole * 2

    opx_Al2O3 = entries[2][1].value
    if opx_Al2O3:
        opx_Al2O3 = float(opx_Al2O3)
        opx_Al2O3_mole = opx_Al2O3 / 101.961276
        opx_Al2O3_cations = opx_Al2O3_mole * 2   
        opx_Al2O3_oxygens = opx_Al2O3_mole * 3

    opx_Cr2O3 = entries[3][1].value
    if opx_Cr2O3:
        opx_Cr2O3 = float(opx_Cr2O3)
        opx_Cr2O3_mole = opx_Cr2O3 / 151.9904
        opx_Cr2O3_cations = opx_Cr2O3_mole * 2   
        opx_Cr2O3_oxygens = opx_Cr2O3_mole * 3
    
    opx_FeO = entries[4][1].value
    if opx_FeO:
        opx_FeO = float(opx_FeO)
        opx_FeO_mole = opx_FeO / 71.8444
        opx_FeO_cations = opx_FeO_mole * 1   
        opx_FeO_oxygens = opx_FeO_mole * 1

    opx_MnO = entries[5][1].value
    if opx_MnO:
        opx_MnO = float(opx_MnO)
        opx_MnO_mole = opx_MnO / 70.937449
        opx_MnO_cations = opx_MnO_mole * 1   
        opx_MnO_oxygens = opx_MnO_mole * 1

    opx_MgO = entries[6][1].value
    if opx_MgO:
        opx_MgO = float(opx_MgO)
        opx_MgO_mole = opx_MgO / 40.3044
        opx_MgO_cations = opx_MgO_mole * 1   
        opx_MgO_oxygens = opx_MgO_mole * 1   

    opx_CaO = entries[7][1].value
    if opx_CaO:
        opx_CaO = float(opx_CaO)
        opx_CaO_mole = opx_CaO / 56.0774
        opx_CaO_cations = opx_CaO_mole * 1   
        opx_CaO_oxygens = opx_CaO_mole * 1   

    opx_Na2O = entries[8][1].value
    if opx_Na2O:
        opx_Na2O = float(opx_Na2O)
        opx_Na2O_mole = opx_Na2O / 61.97894
        opx_Na2O_cations = opx_Na2O_mole * 2   
        opx_Na2O_oxygens = opx_Na2O_mole * 1    
    
    #OLIVINE CALCULATIONS
    ol_SiO2 = entries[0][2].value
    if ol_SiO2:
        ol_SiO2 = float(ol_SiO2)
        ol_SiO2_mole = ol_SiO2 / 60.0843
        ol_SiO2_cations = ol_SiO2_mole * 1
        ol_SiO2_oxygens = ol_SiO2_mole * 2
        
    ol_TiO2 = entries[1][2].value
    if ol_TiO2:
        ol_TiO2 = float(ol_TiO2)
        ol_TiO2_mole = ol_TiO2 / 78.8658
        ol_TiO2_cations = ol_TiO2_mole * 1   
        ol_TiO2_oxygens = ol_TiO2_mole * 2

    ol_Al2O3 = entries[2][2].value
    if ol_Al2O3:
        ol_Al2O3 = float(ol_Al2O3)
        ol_Al2O3_mole = ol_Al2O3 / 101.961276
        ol_Al2O3_cations = ol_Al2O3_mole * 2   
        ol_Al2O3_oxygens = ol_Al2O3_mole * 3

    ol_Cr2O3 = entries[3][2].value
    if ol_Cr2O3:
        ol_Cr2O3 = float(ol_Cr2O3)
        ol_Cr2O3_mole = ol_Cr2O3 / 151.9904
        ol_Cr2O3_cations = ol_Cr2O3_mole * 2   
        ol_Cr2O3_oxygens = ol_Cr2O3_mole * 3
    
    ol_FeO = entries[4][2].value
    if ol_FeO:
        ol_FeO = float(ol_FeO)
        ol_FeO_mole = ol_FeO / 71.8444
        ol_FeO_cations = ol_FeO_mole * 1   
        ol_FeO_oxygens = ol_FeO_mole * 1

    ol_MnO = entries[5][2].value
    if ol_MnO:
        ol_MnO = float(ol_MnO)
        ol_MnO_mole = ol_MnO / 70.937449
        ol_MnO_cations = ol_MnO_mole * 1   
        ol_MnO_oxygens = ol_MnO_mole * 1

    ol_MgO = entries[6][2].value
    if ol_MgO:
        ol_MgO = float(ol_MgO)
        ol_MgO_mole = ol_MgO / 40.3044
        ol_MgO_cations = ol_MgO_mole * 1   
        ol_MgO_oxygens = ol_MgO_mole * 1   

    ol_CaO = entries[7][2].value
    if ol_CaO:
        ol_CaO = float(ol_CaO)
        ol_CaO_mole = ol_CaO / 56.0774
        ol_CaO_cations = ol_CaO_mole * 1   
        ol_CaO_oxygens = ol_CaO_mole * 1   

    ol_P2O5 = entries[8][2].value
    if ol_P2O5:
        ol_P2O5 = float(ol_P2O5)
        ol_P2O5_mole = ol_P2O5 / 141.944522
        ol_P2O5_cations = ol_P2O5_mole * 2   
        ol_P2O5_oxygens = ol_P2O5_mole * 5
        
    #NORMALIZATION CALCULATIONS
    #spl_total_oxygens = spl_SiO2_oxygens + spl_TiO2_oxygens + spl_Al2O3_oxygens + spl_Cr2O3_oxygens + spl_FeO_oxygens +spl_MnO_oxygens + spl_MgO_oxygens + spl_CaO_oxygens + spl_Fe2O3_oxygens + spl_V2O3_oxygens
    #spl_oxygen_factor = 4 / spl_total_oxygens
    #spl_SiO2_cations_normalized = spl_SiO2_cations * spl_oxygen_factor
    #spl_TiO2_cations_normalized = spl_TiO2_cations * spl_oxygen_factor
    #spl_Al2O3_cations_normalized = spl_Al2O3_cations * spl_oxygen_factor
    #spl_Cr2O3_cations_normalized = spl_Cr2O3_cations * spl_oxygen_factor
    #spl_FeO_cations_normalized = spl_FeO_cations * spl_oxygen_factor
    #spl_MnO_cations_normalized = spl_MnO_cations * spl_oxygen_factor
    #spl_MgO_cations_normalized = spl_MgO_cations * spl_oxygen_factor
    #spl_CaO_cations_normalized = spl_CaO_cations * spl_oxygen_factor
    #spl_Fe2O3_cations_normalized = spl_Fe2O3_cations * spl_oxygen_factor
    #spl_V2O3_cations_normalized = spl_V2O3_cations * spl_oxygen_factor
    #spl_total_cations_normalized = spl_SiO2_cations_normalized + spl_TiO2_cations_normalized + spl_Al2O3_cations_normalized + spl_Cr2O3_cations_normalized + spl_FeO_cations_normalized + spl_MnO_cations_normalized + spl_MgO_cations_normalized + spl_CaO_cations_normalized + spl_Fe2O3_cations_normalized + spl_V2O3_cations_normalized

    spl_total_cations = spl_SiO2_cations + spl_TiO2_cations + spl_Al2O3_cations + spl_Cr2O3_cations + spl_FeO_cations +spl_MnO_cations + spl_MgO_cations + spl_CaO_cations + spl_Fe2O3_cations + spl_V2O3_cations
    spl_oxygen_factor = 4 / spl_total_cations
    spl_SiO2_cations_normalized = spl_SiO2_cations * spl_oxygen_factor
    spl_TiO2_cations_normalized = spl_TiO2_cations * spl_oxygen_factor
    spl_Al2O3_cations_normalized = spl_Al2O3_cations * spl_oxygen_factor
    spl_Cr2O3_cations_normalized = spl_Cr2O3_cations * spl_oxygen_factor
    spl_FeO_cations_normalized = spl_FeO_cations * spl_oxygen_factor
    spl_MnO_cations_normalized = spl_MnO_cations * spl_oxygen_factor
    spl_MgO_cations_normalized = spl_MgO_cations * spl_oxygen_factor
    spl_CaO_cations_normalized = spl_CaO_cations * spl_oxygen_factor
    spl_Fe2O3_cations_normalized = spl_Fe2O3_cations * spl_oxygen_factor
    spl_V2O3_cations_normalized = spl_V2O3_cations * spl_oxygen_factor
    spl_total_cations_normalized = spl_SiO2_cations_normalized + spl_TiO2_cations_normalized + spl_Al2O3_cations_normalized + spl_Cr2O3_cations_normalized + spl_FeO_cations_normalized + spl_MnO_cations_normalized + spl_MgO_cations_normalized + spl_CaO_cations_normalized + spl_Fe2O3_cations_normalized + spl_V2O3_cations_normalized    
    
    opx_total_valences = (4 * (opx_SiO2_cations + opx_TiO2_cations)) + (3 * (opx_Al2O3_cations + opx_Cr2O3_cations)) + (2 * (opx_MgO_cations + opx_CaO_cations + opx_MnO_cations + opx_FeO_cations)) + (opx_Na2O_cations)
    opx_normalization_factor = 12 / opx_total_valences
    opx_SiO2_cations_normalized = opx_SiO2_cations * opx_normalization_factor
    opx_TiO2_cations_normalized = opx_TiO2_cations * opx_normalization_factor
    opx_Al2O3_cations_normalized = opx_Al2O3_cations * opx_normalization_factor
    opx_Cr2O3_cations_normalized = opx_Cr2O3_cations * opx_normalization_factor
    opx_FeO_cations_normalized = opx_FeO_cations * opx_normalization_factor
    opx_MnO_cations_normalized = opx_MnO_cations * opx_normalization_factor
    opx_MgO_cations_normalized = opx_MgO_cations * opx_normalization_factor
    opx_CaO_cations_normalized = opx_CaO_cations * opx_normalization_factor
    opx_Na2O_cations_normalized = opx_Na2O_cations * opx_normalization_factor
    opx_total_cations_normalized = opx_SiO2_cations_normalized + opx_TiO2_cations_normalized + opx_Al2O3_cations_normalized + opx_Cr2O3_cations_normalized + opx_FeO_cations_normalized + opx_MnO_cations_normalized + opx_MgO_cations_normalized + opx_CaO_cations_normalized + opx_Na2O_cations_normalized
    
    ol_total_cations = ol_SiO2_cations + ol_TiO2_cations + ol_Al2O3_cations + ol_Cr2O3_cations + ol_FeO_cations + ol_MnO_cations + ol_MgO_cations + ol_CaO_cations
    ol_total_valences = (5 * (ol_P2O5_oxygens)) + (4 * (ol_SiO2_oxygens + ol_TiO2_oxygens)) + (3 * (ol_Al2O3_oxygens + ol_Cr2O3_oxygens)) + (2 * (ol_MgO_oxygens + ol_CaO_oxygens + ol_MnO_oxygens + ol_FeO_oxygens))
    ol_normalization_factor = 8 / ol_total_valences
    ol_SiO2_cations_normalized = ol_SiO2_cations * ol_normalization_factor
    ol_TiO2_cations_normalized = ol_TiO2_cations * ol_normalization_factor
    ol_Al2O3_cations_normalized = ol_Al2O3_cations * ol_normalization_factor
    ol_Cr2O3_cations_normalized = ol_Cr2O3_cations * ol_normalization_factor
    ol_FeO_cations_normalized = ol_FeO_cations * ol_normalization_factor
    ol_MnO_cations_normalized = ol_MnO_cations * ol_normalization_factor
    ol_MgO_cations_normalized = ol_MgO_cations * ol_normalization_factor
    ol_CaO_cations_normalized = ol_CaO_cations * ol_normalization_factor
    ol_P2O5_cations_normalized = ol_P2O5_cations * ol_normalization_factor
    ol_total_cations_normalized = ol_SiO2_cations_normalized + ol_TiO2_cations_normalized + ol_Al2O3_cations_normalized + ol_Cr2O3_cations_normalized + ol_FeO_cations_normalized + ol_MnO_cations_normalized + ol_MgO_cations_normalized + ol_CaO_cations_normalized + ol_P2O5_cations_normalized

    # Spinel-Olivine geothermometry
    #molar fraction of Mg and Fe in olivine
    ol_Mg_mole_fraction = ol_MgO_cations_normalized / (ol_MgO_cations_normalized + ol_FeO_cations_normalized)
    ol_Fe_mole_fraction = ol_FeO_cations_normalized / (ol_MgO_cations_normalized + ol_FeO_cations_normalized)

    #molar fraction of Mg, Fe, and Cr in spinel
    spl_Mg_mole_fraction = spl_MgO_cations_normalized / (spl_MgO_cations_normalized + spl_FeO_cations_normalized)
    spl_Fe_mole_fraction = spl_FeO_cations_normalized / (spl_MgO_cations_normalized + spl_FeO_cations_normalized)
    spl_Fe3_mole_fraction = spl_Fe2O3_cations_normalized / (spl_Fe2O3_cations_normalized + spl_Cr2O3_cations_normalized + spl_Al2O3_cations_normalized)
    spl_Cr_mole_fraction = spl_Cr2O3_cations_normalized / (spl_Fe2O3_cations_normalized + spl_Cr2O3_cations_normalized + spl_Al2O3_cations_normalized)

    #natural log
    math.e
    e = math.e
    math.log(e)

    #base lnKd value
    spl_ol_mole_fractions = ((ol_Mg_mole_fraction * spl_Fe_mole_fraction) / (ol_Fe_mole_fraction * spl_Mg_mole_fraction))
    base_ln_Kd = math.log((spl_ol_mole_fractions))

    #ln Kd 0 value
    ln_Kd_zero = base_ln_Kd - (2 * spl_Fe3_mole_fraction)

    #temperature calculation finally :)

    temperature_kelvin = ((4299 * spl_Cr_mole_fraction) + 1283) / ((ln_Kd_zero + (1.469 * spl_Cr_mole_fraction) + 0.363))
    temperature_celcius = temperature_kelvin - 273.15

    #print("Temperature in Celcius using olivine-spinel geothermometer based on work by Jianping et al. (1995), Sack and Ghiorso, (1991), and Fabries (1987)")
    #print(temperature_celcius)

    #OLIVINE-ORTHOPYROXENE-SPINEL OXYBAROMETRY SECTION
    pressure_in_bars_initial = pressure_data[0]
    pressure_in_bars = int(pressure_in_bars_initial)
   
    #spl_a = (math.log((spl_FeO_cations_normalized * (spl_Fe2O3_cations_normalized * spl_Fe2O3_cations_normalized))/4))
    #spl_b = (1/temperature_kelvin) * (406 * (spl_Al2O3_cations_normalized * spl_Al2O3_cations_normalized))
    #spl_c = (653 * spl_MgO_cations_normalized * spl_Al2O3_cations_normalized)
    #spl_d = (299 * (spl_Cr2O3_cations_normalized * spl_Cr2O3_cations_normalized))
    #spl_e = (199 * spl_Al2O3_cations_normalized * spl_Cr2O3_cations_normalized)
    #spl_f = (346 * spl_MgO_cations_normalized * spl_Cr2O3_cations_normalized)
    #intermediate_value_spinel_activity = spl_a + spl_b + spl_c + spl_d + spl_e + spl_f
    #spinel_activity = 2 * (math.log10(intermediate_value_spinel_activity))
    
    Spl = ObjCClass('SpinelBerman')
    obj = Spl.alloc().init()
    print (obj.phaseName)
    
    modelDB = thermo.model.Database()
    SplS = modelDB.get_phase('SplS')    
    data = [spl_SiO2, spl_TiO2, spl_Al2O3, spl_Fe2O3, spl_FeO, spl_MnO, spl_MgO, spl_CaO, spl_Na2O, spl_Cr2O3]

    oxide_names = ['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'Cr2O3']
    def get_oxide_comp(data):
        SplS_mol_oxides = thermo.chem.format_mol_oxide_comp(dict(zip(oxide_names, data)), convert_grams_to_moles=True)
        return SplS_mol_oxides
    def validate_endmember_comp(moles_end, phase):
        print(phase.props['phase_name'])
        sum = 0.0
               
    SplS_mol_oxides = get_oxide_comp(data)
    SplS_moles_end = SplS.calc_endmember_comp(mol_oxide_comp=SplS_mol_oxides, method='intrinsic', normalize=True)
    validate_endmember_comp(SplS_moles_end, SplS)
   
    #calculating the number of components and species
    nc = obj.numberOfSolutionComponents()
    #print ('Number of components = ', nc)
    ns = obj.numberOfSolutionSpecies()
    #print ('Number of species = ', ns)

    PhaseBase = ObjCClass('PhaseBase')
    #print ('component name, formula, and molecular weight (g/mol)')
    for i in range(0, nc):
        component = obj.componentAtIndex_(i)

    chromite = SplS_moles_end[0]
    hercynite = SplS_moles_end[1]
    magnetite = SplS_moles_end[2]
    spinel = SplS_moles_end[3]
    ulvospinel = SplS_moles_end[4]
    
    import ctypes
    m = (ctypes.c_double*nc)()
    ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
    m[0] = chromite
    m[1] = hercynite
    m[2] = magnetite
    m[3] = spinel
    m[4] = ulvospinel
    
    for i in range (0, nc):
        component = obj.componentAtIndex_(2)
        print ('moles of (', component.phaseName.ljust(20), ') = ', m[2])

    t = temperature_kelvin
    p = pressure_in_bars
    activity = obj.getActivityFromMolesOfComponents_andT_andP_(m, t, p)
    potential = obj.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)  
    print (component.phaseName, activity.valueAtIndex_(3), potential.valueAtIndex_(3))
    activity_of_magnetite_component = activity.valueAtIndex_(3)
    print(activity_of_magnetite_component)
    spinel_activity = 2 * (math.log10(activity_of_magnetite_component))
    print(spinel_activity)
    spinel_barometer_calculation = spinel_activity


    #Orthopyroxene calculation section
    opx_Al_IV = 2 - opx_SiO2_cations_normalized
    opx_Al_VI = opx_Al2O3_cations_normalized - opx_Al_IV
    M1_site = opx_Al_VI + opx_Cr2O3_cations_normalized + opx_TiO2_cations_normalized
    M2_site = opx_CaO_cations_normalized + opx_MnO_cations_normalized
    M1_site_Mg_Fe = M1_site + (opx_MgO_cations_normalized /2) + (opx_FeO_cations_normalized /2)
    
    M1_Fe_fraction = (opx_FeO_cations_normalized/2) /  M1_site_Mg_Fe
    
    M2_site_Mg_Fe = M2_site + (opx_MgO_cations_normalized /2) + (opx_FeO_cations_normalized /2)
    
    M2_Fe_fraction = (opx_FeO_cations_normalized/2) /  M2_site_Mg_Fe
    
    opx_M_sites =  (M1_Fe_fraction) *  (M2_Fe_fraction)
    opx_barometer_calculation = (3 * math.log10(opx_M_sites))

    #Olivine calculation section

    ol_barometer_intermediate_calculation = ((2620/temperature_kelvin) * (ol_Mg_mole_fraction * ol_Mg_mole_fraction))
    ol_barometer_calculation = ((12 * (math.log10(ol_Fe_mole_fraction))) - (ol_barometer_intermediate_calculation))

    logfo2calc = ((-24441.9 / temperature_kelvin) + (8.29)) 
    pressure_in_bars_initial = pressure_data[0]
    pressure_in_bars = int(pressure_in_bars_initial)
    log_calc_with_pressure = -logfo2calc + (220/temperature_kelvin) + (0.35) - ((0.0369 * pressure_in_bars)/temperature_kelvin)
    calc_with_pressure = ((220/temperature_kelvin) + (0.35) - ((0.0369 * pressure_in_bars)/temperature_kelvin)) - (ol_barometer_calculation) + (opx_barometer_calculation) + (spinel_barometer_calculation)

    #log fo2 FINALLY!
    #base log is the absolute log
    base_log_fo2 = -1 * (log_calc_with_pressure - (ol_barometer_calculation) + (opx_barometer_calculation) + (spinel_barometer_calculation))
    
    #Frost 1991 values
    Frost_QFM_buffer = (-25096.3/temperature_kelvin) + (8.735) + ((0.110*((pressure_in_bars - 1)))/temperature_kelvin)
    
    #Euster and Wones 1962 values for the Iron_Wustite buffer
    EW62_IW_buffer = (-27215/temperature_kelvin) + (6.57) + ((0.056*((pressure_in_bars - 1)))/temperature_kelvin)
    
    #Huebner and Sato 1970 values for the nickle nickle oxide buffer
    HS70_NNO_buffer = (-24930/temperature_kelvin) + (9.36) + ((0.046*((pressure_in_bars - 1)))/temperature_kelvin)
    
    dqfm_log_fo2 = base_log_fo2 - Frost_QFM_buffer
    diw_log_fo2 = base_log_fo2 - EW62_IW_buffer
    dnno_log_fo2 = base_log_fo2 - HS70_NNO_buffer
    
    log_fo2_QFM = base_log_fo2
    print(dqfm_log_fo2)
    #widget values
    x_widget.value = log_fo2_QFM
    y_widget.value = temperature_celcius
    z_widget.value = dqfm_log_fo2
    aa_widget.value = diw_log_fo2
    ab_widget.value = dnno_log_fo2
    
pass
continue_button.on_click(on_continue_clicked)

# create an output widget
output = widgets.Output()

# display the widget
output

# add some print statements
with output:
    print("")
    print("References: ")
    print("")
    print("Herd, C.D.K., 2008. Basalts as Probes of Planetary Interior Redox State. Reviews in Mineralogy and Geochemistry 68, 527–553. https://doi.org/10.2138/rmg.2008.68.19")
    print("")
    print("Jianping, L., Kornprobst, J., Vielzeuf, D., Fabriès, J., 1995. An improved experimental calibration of the olivine-spinel geothermometer. Chin. J. of Geochem. 14, 68–77. https://doi.org/10.1007/BF02840385")
    print("")
    print("Myers, J., Eugster, H.P., 1983. The system Fe-Si-O: Oxygen buffer calibrations to 1,500K. Contr. Mineral. and Petrol. 82, 75–90. https://doi.org/10.1007/BF00371177")
    print("")
    #print("Sack, R.O., Ghiorso, M.S., 1994a. Thermodynamics of multicomponent pyroxenes: I. Formulation of a general model. Contr. Mineral. and Petrol. 116, 277–286. https://doi.org/10.1007/BF00306497")
    #print("")
    #print("Sack, R.O., Ghiorso, M.S., 1994b. Thermodynamics of multicomponent pyroxenes: II. Phase relations in the quadrilateral. Contr. Mineral. and Petrol. 116, 287–300. https://doi.org/10.1007/BF00306498")
    #print("")
    #print("Sack, R.O., Ghiorso, M.S., 1994c. Thermodynamics of multicomponent pyroxenes: III. Calibration of Fe2+(Mg)-1, TiAl2(MgSi2)-1, TiFe23+(MgSi2)-1, AlFe3+(MgSi)-1, NaAl(CaMg)-1, Al2(MgSi)-1 and Ca(Mg)-1 exchange reactions between pyroxenes and silicate melts. Contr. Mineral. and Petrol. 118, 271–296. https://doi.org/10.1007/BF00306648")
    #print("")
    print("Sack, R.O., Ghiorso, M.S., 1991a. Chromian spinels as petrogenetic indicators: Thermodynamics and petrological applications. American Mineralogist 76, 827–847.")
    print("")
    print("Sack, R.O., Ghiorso, M.S., 1991b. An internally consistent model for the thermodynamic properties of Fe−Mg-titanomagnetite-aluminate spinels. Contr. Mineral. and Petrol. 106, 474–505. https://doi.org/10.1007/BF00321989")
    print("")
    print("Sack, R.O., Ghiorso, M.S., 1989. Importance of considerations of mixing properties in establishing an internally consistent thermodynamic database: thermochemistry of minerals in the system Mg2SiO4-Fe2SiO4-SiO2. Contr. Mineral. and Petrol. 102, 41–68. https://doi.org/10.1007/BF01160190")
    print("")
    print("Wood, B.J., 1990. An experimental test of the spinel peridotite oxygen barometer. Journal of Geophysical Research: Solid Earth 95, 15845–15851. https://doi.org/10.1029/JB095iB10p15845")
    print("")
    print("Wood, B.J., Nell, J., 1991. High-temperature electrical conductivity of the lower-mantle phase (Mg, Fe)O. Nature 351, 309–311. https://doi.org/10.1038/351309a0")


# Create the AppLayout
app = AppLayout(header=None,
                left_sidebar=None,
                center=window,
                right_sidebar=result_box,
                footer=output,
                pane_widths=[10, 10, 10],
                pane_heights=[2, 5, '2000px'])

# Display the AppLayout
app








Welcome! This notebook calculates the oxygen fugacity of a sample using the olivine-orthopyroxene-spinel geo-oxybarometer, and the temperature of Fe-Mg equilibration between olivine and spinel using the olivine-spinel geothermometer. To use this notebook, enter the pressure (in bars) of sample formation, as well as the wt.% values of the listed oxide phases for spinel, orthopyroxene, and olivine.

The oxygen fugacity results include:

logfO2 = absolute log fO2

dQFM fO2 = fO2 relative to the Quartz-Fayalite-Magnetite buffer, with values from Frost (1991)

dIW fO2 = fO2 relative to the Iron-Wustite buffer, with values from Euster and Wones (1962)

dNNO fO2 = fO2 relative to the Nickle-Nickle-Oxide buffer, with values from Huebner and Sato (1970)

The geothermometry and geobarometry calculations in this notebook are based on work by Jianping et al. (1995), Myers and Eugster (1983), Sack and Ghiorso (1989, 1991a, 1991b), Wood (1990), Wood and Nell (1991). For a review on oxybarometers, in

Button(button_style='info', description='Olivine-Pyroxene-Spinel geobarometer', layout=Layout(height='auto', w…

AppLayout(children=(Output(layout=Layout(grid_area='footer')), VBox(children=(FloatText(value=0.0, description…